In [31]:
from os import walk
import cv2
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn.metrics import confusion_matrix

In [32]:
#change the paths to the location of your images
lab = "~/Labridae"
gob = "~/Gobiidae"

fish_paths = []

for (dirpath, dirnames, filenames) in walk(lab):
    for i in range(len(filenames)):
        lab_img = dirpath + '/' + filenames[i]
        if "Store" not in lab_img:  
            fish_paths.append(lab_img)
    break

for (dirpath, dirnames, filenames) in walk(gob):
    for i in range(len(filenames)):
        gob_img = dirpath + '/' + filenames[i]
        if "Store" not in gob_img:
            fish_paths.append(gob_img)
    break

In [33]:
x_train_ordered = []
y_train_ordered = []

for i in fish_paths:
    img = cv2.imread(i)
    # standardize size for prediction step
    img = cv2.resize(img, (150, 150))
    img = img/255.0
    #img = img.reshape((1,) + img.shape)
    x_train_ordered.append(img)

    if "Labridae" in i:
        y_train_ordered.append(0)
    else:
        y_train_ordered.append(1)

In [35]:
to_shuffle = range(len(x_train_ordered))
random.shuffle(to_shuffle)

x_train = []
y_train = []

for i in to_shuffle:
    x_train.append(x_train_ordered[i])
    y_train.append(y_train_ordered[i])

In [37]:
test_indices = sorted(random.sample(range(len(x_train)), 200),reverse=True)

x_test = []
y_test = []

for i in test_indices:
    x_test.append(x_train[i])
    y_test.append(y_train[i])
    
    del x_train[i]
    del y_train[i]

In [38]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [48]:
# dimensions of our images.
#img_width, img_height = 150, 150

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [49]:
model.fit(x_train, y_train, 
          batch_size=32, epochs=7, verbose=1, validation_data = (x_test,y_test))

Train on 1797 samples, validate on 200 samples
Epoch 1/7
1797/1797 [==============================] - 80s 45ms/step - loss: 0.1299 - acc: 0.8191 - val_loss: 0.0710 - val_acc: 0.8950
Epoch 2/7
1797/1797 [==============================] - 80s 44ms/step - loss: 0.0458 - acc: 0.9438 - val_loss: 0.0215 - val_acc: 0.9700
Epoch 3/7
1797/1797 [==============================] - 93s 52ms/step - loss: 0.0279 - acc: 0.9677 - val_loss: 0.0662 - val_acc: 0.9150
Epoch 4/7
1797/1797 [==============================] - 83s 46ms/step - loss: 0.0273 - acc: 0.9672 - val_loss: 0.0235 - val_acc: 0.9700
Epoch 5/7
1797/1797 [==============================] - 91s 50ms/step - loss: 0.0198 - acc: 0.9738 - val_loss: 0.0158 - val_acc: 0.9850
Epoch 6/7
1797/1797 [==============================] - 82s 45ms/step - loss: 0.0137 - acc: 0.9833 - val_loss: 0.0189 - val_acc: 0.9800
Epoch 7/7
1797/1797 [==============================] - 77s 43ms/step - loss: 0.0170 - acc: 0.9794 - val_loss: 0.0136 - val_acc: 0.9850


In [42]:
predictions = model.predict(x_test).round()

In [66]:
confusion_matrix(y_test, np.squeeze(predictions, axis=1))

array([[113,   1],
       [  2,  84]])